In [21]:
!curl -L -o train.tar.gz https://www.dropbox.com/s/5jv7glyhgzps768/train.tar.gz?dl=0

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1034    0  1034    0     0    989      0 --:--:--  0:00:01 --:--:--     0
100 15.2M  100 15.2M    0     0  9814k      0  0:00:01  0:00:01 --:--:-- 9814k


In [23]:
!tar -xzvf train.tar.gz

train.csv


In [31]:
!pwd

/datadisk/work/blazing_poc/conduit_blazing_demo


In [24]:
import os
import sys
import psutil
import pyarrow.parquet as pq
import time
import cudf
import subprocess
from subprocess import Popen, PIPE
from blazingsql import BlazingContext
import numpy as np
import pandas as pd


In [30]:

INPUT_FOLDER=os.getcwd()
print(INPUT_FOLDER)

bc = BlazingContext(allocator="managed", #default
                        pool=False, #true
                        initial_pool_size=None,
                        enable_logging=False)



/datadisk/work/blazing_poc/conduit_blazing_demo
BlazingContext ready


Read everything in cudf - dataframes backed by gpu



*Read from cudf all the data*

In [32]:

train_df=cudf.read_csv(os.path.join(INPUT_FOLDER,"train.csv"))
train_df.to_parquet(os.path.join(INPUT_FOLDER,"train.parquet"))



**Here we create the tables in blazing**


In [33]:

bc.create_table("train",[os.path.join(INPUT_FOLDER,"train.parquet")])
bc.create_table("traincu",train_df)



file /datadisk/work/blazing_poc/conduit_blazing_demo/train.parquet


In [34]:
bc.sql("select count(*) from train")

,COUNT(*)
0,1999999


In [35]:
bc.sql("select count(*) from traincu")

,COUNT(*)
0,1999999


In [36]:
bc.sql("select * from train limit 10")

,Unnamed: 0,week,sales_depot_id,sales_channel_id,route_id,client_id,product_id,sales_week_units,sales_value,return_units_next_week,returns_value_next_week,demand
0,0,3,1130,1,1217,15766,1212,3,25.14,0,0.0,3
1,1,3,1130,1,1217,15766,1216,4,33.52,0,0.0,4
2,2,3,1130,1,1217,15766,1238,4,39.32,0,0.0,4
3,3,3,1130,1,1217,15766,1240,4,33.52,0,0.0,4
4,4,3,1130,1,1217,15766,1242,3,22.92,0,0.0,3
5,5,3,1130,1,1217,15766,1250,5,38.20,0,0.0,5
6,6,3,1130,1,1217,15766,1309,3,20.28,0,0.0,3
7,7,3,1130,1,1217,15766,3894,6,56.10,0,0.0,6
8,8,3,1130,1,1217,15766,4085,4,24.60,0,0.0,4
9,9,3,1130,1,1217,15766,5310,6,31.68,0,0.0,6


In [37]:
bc.sql("select * from traincu limit 10")

,Unnamed: 0,week,sales_depot_id,sales_channel_id,route_id,client_id,product_id,sales_week_units,sales_value,return_units_next_week,returns_value_next_week,demand
0,0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3
5,5,3,1110,7,3301,15766,1250,5,38.20,0,0.0,5
6,6,3,1110,7,3301,15766,1309,3,20.28,0,0.0,3
7,7,3,1110,7,3301,15766,3894,6,56.10,0,0.0,6
8,8,3,1110,7,3301,15766,4085,4,24.60,0,0.0,4
9,9,3,1110,7,3301,15766,5310,6,31.68,0,0.0,6


In [38]:
res = bc.sql("select sales_depot_id,count(distinct route_id) as feat_sales_depot_id_cnt_route_id from train group by sales_depot_id order by feat_sales_depot_id_cnt_route_id desc")
res

,sales_depot_id,feat_sales_depot_id_cnt_route_id
0,1119,406
1,1123,194
2,1121,132
3,1126,130
4,1142,129
5,1116,127
6,1114,126
7,1122,119
8,1111,118
9,1127,111


In [39]:
res = bc.sql("select sales_depot_id,count(distinct route_id) as feat_sales_depot_id_cnt_route_id from traincu group by sales_depot_id order by feat_sales_depot_id_cnt_route_id desc")
res

,sales_depot_id,feat_sales_depot_id_cnt_route_id
0,1167,195
1,1123,194
2,1220,165
3,1219,158
4,1130,145
5,1137,138
6,1121,135
7,1117,133
8,1168,131
9,1126,130


In [40]:
feature_table_name = "table1"
os.path.join(INPUT_FOLDER,feature_table_name+".parquet")

'/datadisk/work/blazing_poc/conduit_blazing_demo/table1.parquet'

In [41]:
feature_table_name = "table1"
feature_table_name_cu = "tablecu1"
res = bc.sql("select sales_depot_id,count(distinct route_id) as feat_sales_depot_id_cnt_route_id from train group by sales_depot_id order by feat_sales_depot_id_cnt_route_id desc")

res.to_parquet(os.path.join(INPUT_FOLDER,feature_table_name+".parquet"))
parquet_file=[os.path.join(INPUT_FOLDER,feature_table_name+".parquet")]
bc.create_table(feature_table_name,parquet_file)
print("[Parquet]Created:",feature_table_name)

bc.create_table(feature_table_name_cu,res)
print("[CUDF]Created:",feature_table_name_cu)




file /datadisk/work/blazing_poc/conduit_blazing_demo/table1.parquet
[Parquet]Created: table1
[CUDF]Created: tablecu1


In [42]:
bc.sql("select sales_depot_id,count(distinct client_id) as feat_sales_depot_id_cnt_client_id from train group by sales_depot_id order by feat_sales_depot_id_cnt_client_id desc")


,sales_depot_id,feat_sales_depot_id_cnt_client_id
0,1111,9656
1,1127,9209
2,1117,8868
3,1126,7323
4,1121,7119
5,1116,6957
6,1123,6776
7,1124,6251
8,1122,6099
9,1119,4317


In [43]:
feature_table_name = "table2"
feature_table_name_cu = "tablecu2"
res= bc.sql("select sales_depot_id,count(distinct client_id) as feat_sales_depot_id_cnt_client_id from train group by sales_depot_id order by feat_sales_depot_id_cnt_client_id desc")


res.to_parquet(os.path.join(INPUT_FOLDER,feature_table_name+".parquet"))
parquet_file=[os.path.join(INPUT_FOLDER,feature_table_name+".parquet")]
bc.create_table(feature_table_name,parquet_file)
print("[Parquet]Created:",feature_table_name)

bc.create_table(feature_table_name_cu,res)
print("[CUDF]Created:",feature_table_name_cu)

file /datadisk/work/blazing_poc/conduit_blazing_demo/table2.parquet
[Parquet]Created: table2
[CUDF]Created: tablecu2


In [44]:
#this is a join with a table created from cuda: traincu
query="""
select a.*,f0.feat_sales_depot_id_cnt_route_id,f1.feat_sales_depot_id_cnt_client_id from traincu a
JOIN table1 f0 on a.sales_depot_id=f0.sales_depot_id
JOIN table2 f1 on a.sales_depot_id=f1.sales_depot_id
"""
res = bc.sql(query)
print(res)

         Unnamed: 0  week  sales_depot_id  sales_channel_id  route_id  \
0              9632     3            1111                 1      1003   
1              9633     3            1111                 1      1003   
2              9634     3            1111                 1      1003   
3              9635     3            1111                 1      1003   
4              9636     3            1111                 1      1003   
...             ...   ...             ...               ...       ...   
1242837     1829854     3            1219                 1      4480   
1242838     1829855     3            1219                 1      4480   
1242839     1829792     3            1219                 1      4480   
1242840     1829793     3            1219                 1      4480   
1242841     1829794     3            1219                 1      4480   

         client_id  product_id  sales_week_units  sales_value  \
0          1252829       46772                 4        35

In [45]:
#this is a join with a table created from parquet: train
query="""
select a.*,f0.feat_sales_depot_id_cnt_route_id,f1.feat_sales_depot_id_cnt_client_id from train a
JOIN table1 f0 on a.sales_depot_id=f0.sales_depot_id
JOIN table2 f1 on a.sales_depot_id=f1.sales_depot_id
"""
res = bc.sql(query)
print(res)

        Unnamed: 0  week sales_depot_id sales_channel_id route_id client_id  \
0            16736     3           1111                1     1017   1446507   
1            16737     3           1111                1     1017   1446507   
2            16738     3           1111                1     1017   1446507   
3            16739     3           1111                1     1017   1446507   
4            16740     3           1111                1     1017   1446507   
...            ...   ...            ...              ...      ...       ...   
1442823       null  null           null             null     null      null   
1442824       null  null           null             null     null      null   
1442825       null  null           null             null     null      null   
1442826       null  null           null             null     null      null   
1442827       null  null           null             null     null      null   

        product_id sales_week_units sales_value ret